# Creating EnzymeML document from NMR pickle data

This prepares the EnzymeML document, adds all the reaction metadata, and populates the individual NMR runs as `measurement`s.

In [1]:
import glob, pickle, os
import numpy as np

from pyenzyme import (
    EnzymeMLDocument,
    Reactant, Vessel,
    Protein, EnzymeReaction,
    KineticModel, Measurement,
    Replicate
)

## Prepare EnzymeML document

add reactants, reactions, vessels, etc.

In [2]:
# Initialize EnzymeMLDocument
enzmldoc = EnzymeMLDocument(name="PGM-ENO")

# Add Vessel
vessel = Vessel(name="NMRtube", volume=1.0, unit="ml")
vessel_id = enzmldoc.addVessel(vessel)

# Add proteins and reactants
p3ga_id = enzmldoc.addReactant(Reactant(
    name = "3-phosphoglycerate", vessel_id = vessel_id, init_conc = 10.0,
    unit = "mmole / l", constant = False, smiles = 'C(C(C(=O)O)O)OP(=O)(O)O',
    chebi_id = 'CHEBI:132960',
))

p2ga_id = enzmldoc.addReactant(Reactant(
    name = "2-phosphoglycerate", vessel_id = vessel_id, init_conc = 0.0001,
    unit = "mmole / l", constant = False, smiles = 'C(C(C(=O)O)OP(=O)(O)O)O',
    chebi_id = 'CHEBI:88350',
))

PEP_id = enzmldoc.addReactant(Reactant(
    name = "phosphoenolpyruvate", vessel_id = vessel_id, init_conc = 0.0001,
    unit = "mmole / l", constant = False, smiles = 'C=C(C(=O)O)OP(=O)(O)O',
    chebi_id = 'CHEBI:18021',
))

prot_id = enzmldoc.addProtein(Protein(
    name="Total lysate protein", sequence = "UNKNOWN", vessel_id=vessel_id, 
    organism="Saccharomyces cerevisiae", init_conc=0.1418, unit="mg / ml",
))

In [3]:
# Create reactions
reaction1 = EnzymeReaction(temperature=25.0, temperature_unit="C", 
                           ph=7.0, name="Phosphoglyerate mutase", reversible=True)
reaction1.addEduct(p3ga_id, 1.0, enzmldoc)
reaction1.addProduct(p2ga_id, 1.0, enzmldoc)
reaction1.addModifier(prot_id, 1.0, enzmldoc, False)

reaction2 = EnzymeReaction(temperature=25.0, temperature_unit="C", 
                           ph=7.0, name="Enolase", reversible=True)
reaction2.addEduct(p2ga_id, 1.0, enzmldoc)
reaction2.addProduct(PEP_id, 1.0, enzmldoc)
reaction2.addModifier(prot_id, 1.0, enzmldoc, False)

## Add Kinetic Models

This is **not needed**. Thus commented out here. We want to "learn" the kinetics through machine learning. If you were to do classical parameter fitting, then this would be added.

In [4]:
# Create models py parsing equations

# kinmod_PGM = KineticModel.fromEquation("RMM_PGM", 
#                             "p0*Vf_pgm/K_pg3_pgm*(s0 - s1/Keq_pgm)/(1 + s0/K_pg3_pgm + s1/K_pg2_pgm)")
# kinmod_ENO = KineticModel.fromEquation("RMM_ENO", 
#                             "p0*Vf_eno/K_pg2_eno*(s1 - s2/Keq_eno)/(1 + s1/K_pg2_eno + s2/K_pep_eno)")

In [5]:
# Configure parameters for models

# kinmod_PGM._Vf_pgm.update(value=9.4, unit="mM / min", lower=1e-9)
# kinmod_PGM._Keq_pgm.update(value=0.19, unit="dimensionless", lower=1e-9)
# kinmod_PGM._K_pg3_pgm.update(value=1.2, unit="mM", lower=1e-9)
# kinmod_PGM._K_pg2_pgm.update(value=0.08, unit="mM", lower=1e-9)

# kinmod_ENO._Vf_eno.update(value=1.35, unit="mM / min", lower=1e-9)
# kinmod_ENO._Keq_eno.update(value=6.7, unit="dimensionless", lower=1e-9)
# kinmod_ENO._K_pg2_eno.update(value=0.04, unit="mM", lower=1e-9)
# kinmod_ENO._K_pep_eno.update(value=0.5, unit="mM", lower=1e-9)

In [6]:
# Add models to each reaction

# reaction1.model = kinmod_PGM
# reaction2.model = kinmod_ENO

## Add reactions to EnzymeML document

In [7]:
# Add reactions to document

enzmldoc.addReactions([reaction1, reaction2])

{'Phosphoglyerate mutase': 'r0', 'Enolase': 'r1'}

## Add Measurements

The pickle directory should contain all the pickles of the datasets you want to add to the document.

In [8]:
def add_pickle_dir_to_enzymemldoc(path, enzmldoc):
    files = glob.glob(path+'/*')
    files.sort()
    print(files)
    for fn in files:
        if fn.endswith('.pickle'):
            dataset = extract_pickle(fn)
            add_measurement_to_enzymeml(dataset, enzmldoc)
        
def extract_pickle(fn):
    assert(fn.endswith('.pickle'))
    with open(fn, 'rb') as f:
        nmr = pickle.load(f)

    rt = nmr['sundry']['rt']
    nmrtime = nmr['sundry']['time']
    # sanity checks
    rt = nmrtime[1] - nmrtime[0]
    assert rt == nmrtime[1] - nmrtime[0]
    assert len(nmrtime) == len(nmr['PEP'])

    # ensure floats
    for k in ['initp3ga', 'initp2ga', 'initPEP']:
        nmr['sundry'][k] = float(nmr['sundry'][k])
    
    # obtain correct time array, compensate for start of experiment
    time_corr = nmrtime 
    nmr['time'] = time_corr
    nmr['filename'] = os.path.split(fn)[-1]
    return nmr

    
    
def add_measurement_to_enzymeml(dataset, enzmldoc):
    measurement = Measurement(name=dataset["filename"][:-8])
    measurement.global_time_unit = "min"
    
    measurement.global_time = dataset["time"].tolist()
    
    # Add individial metadata
    measurement.ph = 7.0
    measurement.temperature = 25
    measurement.temperature_unit = "C"
    measurement.addData(protein_id=prot_id, init_conc=dataset["sundry"]["tp"], unit="mg / ml")
    measurement.addData(reactant_id=p3ga_id, init_conc=dataset["sundry"]["initp3ga"], unit="mmole / l")
    measurement.addData(reactant_id=p2ga_id, init_conc=dataset["sundry"]["initp2ga"], unit="mmole / l")
    measurement.addData(reactant_id=PEP_id, init_conc=dataset["sundry"]["initPEP"], unit="mmole / l")
    
    # Add replicates
    
    for species_id in ["p3ga", "p2ga", "PEP"]:
        replicate = Replicate(
            id=species_id, species_id=eval(f"{species_id}_id"),
            data_unit="mmole / l", time_unit="min",
            data=dataset[species_id].tolist(), time=measurement.global_time
        )
        measurement.addReplicates(replicate, enzmldoc)
    
    enzmldoc.addMeasurement(measurement)

In [9]:
add_pickle_dir_to_enzymemldoc('Kamogelo_Pickels', enzmldoc)

['Kamogelo_Pickels\\nmr_10mm3pga_10lys_stdpH7_scerev_180821.pickle', 'Kamogelo_Pickels\\nmr_10mmPEP4mm3pga_10lys_stdpH7_scerev_250821.pickle', 'Kamogelo_Pickels\\nmr_10mmPEP_10lys_stdpH7_scerev_190821.pickle', 'Kamogelo_Pickels\\nmr_10mmPEP_10lys_stdpH7_scerev_220821.pickle', 'Kamogelo_Pickels\\nmr_10mmPGA4mm2pep_10lys_stdpH7_scerev_260821.pickle', 'Kamogelo_Pickels\\nmr_12.5mm3pga_10lys_stdpH7_scerev_180821.pickle', 'Kamogelo_Pickels\\nmr_12.5mmPEP_10lys_stdpH7_scerev_190821.pickle', 'Kamogelo_Pickels\\nmr_12.5mmPEP_10lys_stdpH7_scerev_220821.pickle', 'Kamogelo_Pickels\\nmr_15mm3pga_10lys_stdpH7_scerev_180821.pickle', 'Kamogelo_Pickels\\nmr_15mmPEP_10lys_stdpH7_scerev_190821.pickle', 'Kamogelo_Pickels\\nmr_15mmPEP_10lys_stdpH7_scerev_220821.pickle', 'Kamogelo_Pickels\\nmr_17.5mm3pga_10lys_stdpH7_scerev_180821', 'Kamogelo_Pickels\\nmr_17.5mm3pga_10lys_stdpH7_scerev_180821.pickle', 'Kamogelo_Pickels\\nmr_17.5mm3pga_10lys_stdpH7_scerev_180821.zip', 'Kamogelo_Pickels\\nmr_17.5mmPEP_10lys_

In [10]:
enzmldoc.measurement_dict

{'m0': >>> Measurement m0: nmr_10mm3pga_10lys_stdpH7_scerev_18082
     s0 | initial conc: 8.906320330754877 mmole / l 	| #replicates: 1
     s1 | initial conc: 1e-10 mmole / l 	| #replicates: 1
     s2 | initial conc: 1e-10 mmole / l 	| #replicates: 1
     p0 | initial conc: 0.199 mg / ml 	| #replicates: 0,
 'm1': >>> Measurement m1: nmr_10mmPEP4mm3pga_10lys_stdpH7_scerev_25082
     s0 | initial conc: 3.9786725015806583 mmole / l 	| #replicates: 1
     s1 | initial conc: 1e-10 mmole / l 	| #replicates: 1
     s2 | initial conc: 13.107399391768546 mmole / l 	| #replicates: 1
     p0 | initial conc: 0.889 mg / ml 	| #replicates: 0,
 'm2': >>> Measurement m2: nmr_10mmPEP_10lys_stdpH7_scerev_19082
     s0 | initial conc: 1e-10 mmole / l 	| #replicates: 1
     s1 | initial conc: 1e-10 mmole / l 	| #replicates: 1
     s2 | initial conc: 9.5 mmole / l 	| #replicates: 1
     p0 | initial conc: 0.199 mg / ml 	| #replicates: 0,
 'm3': >>> Measurement m3: nmr_10mmPEP_10lys_stdpH7_scerev_22082
   

## Save EnzymeML document

In [11]:
# Finally, write to EnzymeML
enzmldoc.toFile(".", "PGM-ENO")


Archive was written to .\PGM-ENO.omex



In [12]:
pip install pyenzyme --upgrade

Note: you may need to restart the kernel to use updated packages.


In [13]:
enzmldoc.visualize(interactive=True, trendline=False, use_names=True, template="plotly_white")